# Named Entity Recognition Pipeline

El pipeline toma una URL de un feed en formato RSS, obtiene el título y descripción de los artículos en el feed, detecta las NER con un modelo pre-entrenado, y las muestra ordenadas por frecuencia de aparición.

### Versiones
Probado con:
* Almond 0.6.0
* Ammonite 1.6.7
* Scala library version **2.11.12** -- Copyright 2002-2017, LAMP/EPFL
* Java 1.8.0_282

Para ver más información ir a (Help -> About Scala Kernel)

## 1. Obtener texto

### 1.1 Importar librerías

In [22]:
// Equivalent of adding dependencies to maven or sbt files
// For example, to add "org.scalaj" %% "scalaj-http" % "2.4.2" 
import $ivy.`org.scalaj::scalaj-http:2.4.2`
// "org.scala-lang.modules" %% "scala-xml" % "1.3.0"
import $ivy.`org.scala-lang.modules::scala-xml:1.3.0`

import $ivy.$                              
// "org.scala-lang.modules" %% "scala-xml" % "1.3.0"

import $ivy.$                                        

In [23]:
import scalaj.http.{Http, HttpResponse}
import scala.xml.XML

import scalaj.http.{Http, HttpResponse}

import scala.xml.XML

### 1.1 Obtener el texto del RSS Feed

Realizamos una consulta HTTP, que nos devuelve una instancia de HTTPResponse. Dentro del atributo `body` de la HTTPResponse, se encuentra el texto del feed en formato XML. Luego, se parsea el XML para extraer los campos `title` y `description`.

In [24]:
// Tutorial https://alvinalexander.com/source-code/scala-how-to-http-download-xml-rss-feed-timeout/
// get the xml content using scalaj-http
val url = "https://www.chicagotribune.com/arcio/rss/category/sports/?query=display_date:[now-2d+TO+now]&sort=display_date:desc"
val response: HttpResponse[String] = Http(url)
  .timeout(connTimeoutMs = 2000, readTimeoutMs = 5000)
  .asString
val xmlString = response.body
// convert the `String` to a `scala.xml.Elem`
val xml = XML.loadString(xmlString)
// Extract text from title and description
val rssText = (xml \\ "item").map { item =>
    ((item \ "title").text ++ " " ++ (item \ "description").text)
}

url: String = "https://www.chicagotribune.com/arcio/rss/category/sports/?query=display_date:[now-2d+TO+now]&sort=display_date:desc"
response: HttpResponse[String] = HttpResponse(
  """<?xml version="1.0" encoding="utf-8"?>
<rss version="2.0" xmlns:atom="http://www.w3.org/2005/Atom" xmlns:content="http://purl.org/rss/1.0/modules/content/" xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:media="http://search.yahoo.com/mrss/" xmlns:sy="http://purl.org/rss/1.0/modules/syndication/"><channel><title>Chicago Tribune</title><link>https://www.chicagotribune.com</link><language>en-US</language><copyright>© 2021 Chicago Tribune</copyright><atom:link href="https://www.chicagotribune.com/arcio/rss/category/sports/?query=display_date:%5Bnow-2d+TO+now%5D&amp;sort=display_date:desc" rel="self" type="application/rss+xml"/><description>Chicago Tribune News Feed</description><lastBuildDate>Mon, 12 Apr 2021 21:11:37 +0000</lastBuildDate><ttl>1</ttl><sy:updatePeriod>hourly</sy:updatePeriod><sy:updateFrequ

## 2. Detectar las entidades nombradas

### 2.1 Crear el modelo

El **modelo** es sólo la función `getNEs`, que recibe una lista de textos.
Para cada texto, se separa las palabras del texto usando los espacios, y considera que es una entidad nombrada si empieza con mayúscula.

Este código lista los signos de puntuación y algunas palabras comunes del inglés que se van a sacar del texto.

In [25]:
val STOPWORDS = Seq (
    "i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you",
    "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she",
    "her", "hers", "herself", "it", "its", "itself", "they", "them", "your",
    "their", "theirs", "themselves", "what", "which", "who", "whom",
    "this", "that", "these", "those", "am", "is", "are", "was", "were",
    "be", "been", "being", "have", "has", "had", "having", "do", "does",
    "did", "doing", "a", "an", "the", "and", "but", "if", "or",
    "because", "as", "until", "while", "of", "at", "by", "for", "with",
    "about", "against", "between", "into", "through", "during", "before",
    "after", "above", "below", "to", "from", "up", "down", "in", "out",
    "off", "over", "under", "again", "further", "then", "once", "here",
    "there", "when", "where", "why", "how", "all", "any", "both", "each",
    "few", "more", "most", "other", "some", "such", "no", "nor", "not",
    "only", "own", "same", "so", "than", "too", "very", "s", "t", "can",
    "will", "just", "don", "should", "now", "on",
    // Contractions without '
    "im", "ive", "id", "Youre", "youd", "youve",
    "hes", "hed", "shes", "shed", "itd", "were", "wed", "weve",
    "theyre", "theyd", "theyve",
    "shouldnt", "couldnt", "musnt", "cant", "wont",
    // Common uppercase words
    "hi", "hello"
)
val punctuationSymbols = ".,()!?;:'`´\n"
val punctuationRegex = "\\" + punctuationSymbols.split("").mkString("|\\")

STOPWORDS: Seq[String] = List(
  "i",
  "me",
  "my",
  "myself",
  "we",
  "our",
  "ours",
  "ourselves",
  "you",
  "yours",
  "yourself",
  "yourselves",
  "he",
  "him",
  "his",
  "himself",
  "she",
  "her",
  "hers",
  "herself",
  "it",
  "its",
  "itself",
  "they",
  "them",
  "your",
  "their",
  "theirs",
  "themselves",
  "what",
  "which",
  "who",
  "whom",
  "this",
  "that",
  "these",
  "those",
  "am",
...
punctuationSymbols: String = """.,()!?;:'`´
"""
punctuationRegex: String = """\.|\,|\(|\)|\!|\?|\;|\:|\'|\`|\´|\
"""

In [26]:
// Extract Named Entities from a single text
def getNEsSingle(text: String): Seq[String] =
  text.replaceAll(punctuationRegex, "").split(" ")
    .filter { word:String => word.length > 1 &&
              Character.isUpperCase(word.charAt(0)) &&
              !STOPWORDS.contains(word.toLowerCase) }.toSeq

def getNEs(textList: Seq[String]): Seq[Seq[String]] = textList.map(getNEsSingle)

defined function getNEsSingle
defined function getNEs

### 2.2 Aplicar el "Modelo" a los datos

In [27]:
val result = getNEs(rssText)

result: Seq[Seq[String]] = List(
  ArrayBuffer(
    "White",
    "Sox",
    "Tim",
    "Anderson",
    "Celebrating",
    "Tim",
    "Anderson",
    "FOCO"
  ),
  ArrayBuffer(
    "Minnesota",
    "Twins",
    "Wild",
    "Timberwolves",
    "Monday",
    "Minneapolis",
    "Minnesota",
    "Twins",
    "Boston",
    "Red",
    "Sox",
    "Monday",
    "Black",
    "NHL\u2019s",
    "Minnesota",
    "Wild",
    "NBA\u2019s",
    "Minnesota",
    "Timberwolves",
    "Monday"
  ),
  ArrayBuffer(
    "Chicago",
    "Blackhawks",
    "NHL",
    "Adam",
    "Gaudette",
...

## 3. Contar y ordenar las entidades

Concatenar todas las listas, contar cada Named Entity, y luego ordernar por frecuencia

In [28]:
val counts: Map[String, Int] = result.flatten
  .foldLeft(Map.empty[String, Int]) {
     (count, word) => count + (word -> (count.getOrElse(word, 0) + 1)) }
val sortedNEs = counts.toList
  .sortBy(_._2)(Ordering[Int].reverse)

counts: Map[String, Int] = Map(
  "Rate" -> 1,
  "Rapinoe" -> 1,
  "Los" -> 2,
  "Mercedes" -> 3,
  "Park" -> 5,
  "Wolverhampton" -> 1,
  "That\u2019s" -> 1,
  "Yerm\u00edn" -> 3,
  "FOCO" -> 1,
  "Blackhawks\u2019" -> 1,
  "Lynx" -> 2,
  "Sports" -> 2,
  "Gaudette" -> 1,
  "Celebrating" -> 1,
  "Minneapolis" -> 1,
  "Nationwide" -> 1,
  "Alex" -> 2,
  "Japan" -> 1,
  "Lore" -> 1,
  "Angeles" -> 2,
  "Boston" -> 1,
  "Prince" -> 2,
  "Crochet" -> 1,
  "Closer" -> 1,
  "Megan" -> 1,
  "Liam" -> 1,
  "Tigers" -> 1,
  "US" -> 1,
  "Towns" -> 1,
  "Timberwolves" -> 6,
  "Tennessee" -> 2,
  "Minnesota" -> 9,
  "Rodriguez" -> 2,
  "Cubs\u2019" -> 2,
  "WNBA\u2019s" -> 1,
  "Guaranteed" -> 1,
  "Janmark" -> 1,
  "Red" -> 1,
...
sortedNEs: List[(String, Int)] = List(
  ("Chicago", 29),
  ("Sox", 14),
  ("White", 13),
  ("Minnesota", 9),
  ("Pirates", 7),
  ("Cubs", 7),
  ("Pittsburgh", 7),
  ("Timberwolves", 6),
  ("Saturday", 6),
  ("Royals", 6),
  ("City", 6),
  ("Kansas", 6),
  ("Park", 5)